In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
DATASET_PATH = "../dataset.pkl"
IMAGE_SIZE = (128, 128, 3)
NUM_CLASS = 10
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001

In [3]:
def load_dataset(dataset_path):
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    X = dataset['X']
    y = dataset['y']
    class_names = dataset['class_names']
    return X, y, class_names


In [4]:
def preprocess_data(X, y, class_names):
    X = X.astype('float32') / 255.0
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded, num_classes=len(class_names))

    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_categorical, test_size=0.3, random_state=42, stratify=y_categorical
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )
    return X_train, X_val, X_test, y_train, y_val, y_test, label_encoder

In [5]:
base_model = MobileNetV2(
    input_shape=IMAGE_SIZE,
    include_top=False,
    weights='imagenet'
)    
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASS, activation='softmax')
])

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
def train_model(model, X_train, y_train, X_val, y_val):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7
        )
    ]
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        fill_mode='nearest'
    )
    
    print("Début de l'entraînement...")
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    
    return history

In [ ]:
def evaluate_model(model, X_test, y_test, class_names, label_encoder):
    print("Évaluation du modèle...")
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Précision sur le test: {test_accuracy:.4f}")
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    from sklearn.metrics import classification_report, confusion_matrix
    print("\nRapport de classification:")
    print(classification_report(
        true_classes, 
        predicted_classes, 
        target_names=class_names
    ))
    return test_accuracy

In [8]:
X, y, class_names = load_dataset(DATASET_PATH)
X_train, X_val, X_test, y_train, y_val, y_test, label_encoder = preprocess_data(X, y, class_names)

history = train_model(model, X_train, y_train, X_val, y_val)
test_accuracy = evaluate_model(model, X_test, y_test, class_names, label_encoder)

🚀 Début de l'entraînement...
Epoch 1/20


2025-06-26 17:35:39.171529: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2025-06-26 17:35:39.229155: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2025-06-26 17:35:39.240795: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2025-06-26 17:35:39.267788: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50331648 exceeds 10% of free system memory.
2025-06-26 17:35:39.296003: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51916800 exceeds 10% of free system memory.


175/175 [==============================] - 56s 303ms/step - loss: 0.9003 - accuracy: 0.7084 - val_loss: 0.3738 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 2/20
175/175 [==============================] - 46s 265ms/step - loss: 0.5467 - accuracy: 0.8246 - val_loss: 0.2858 - val_accuracy: 0.9050 - lr: 0.0010
Epoch 3/20
175/175 [==============================] - 41s 234ms/step - loss: 0.4870 - accuracy: 0.8407 - val_loss: 0.2768 - val_accuracy: 0.9133 - lr: 0.0010
Epoch 4/20
175/175 [==============================] - 46s 265ms/step - loss: 0.4498 - accuracy: 0.8523 - val_loss: 0.2580 - val_accuracy: 0.9075 - lr: 0.0010
Epoch 5/20
175/175 [==============================] - 40s 229ms/step - loss: 0.4348 - accuracy: 0.8546 - val_loss: 0.2626 - val_accuracy: 0.9167 - lr: 0.0010
Epoch 6/20
175/175 [==============================] - 45s 257ms/step - loss: 0.4080 - accuracy: 0.8643 - val_loss: 0.2756 - val_accuracy: 0.9125 - lr: 0.0010
Epoch 7/20
175/175 [==============================] - 45s 257ms

In [9]:
model.save('../model_classification.h5')

/home/juzoo/Epitech/TEK1/HUB/Anidex/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
